In [1]:
%pip install pandas
%pip install python-dotenv
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas
import requests

import numpy as np

from typing import Final
from dateutil import parser
from functools import  cache
from dotenv import load_dotenv

In [ ]:
load_dotenv()
API_KEY = os.getenv("EXCHANGERATE_API_KEY")
if not API_KEY:
    raise ValueError("Exchange rate API key not found in environment variables. Use EXCHANGERATE_API_KEY.")

In [3]:
path_to_dataset: Final[str] = "synthetic_fraud_data.csv"
exchange_rate_cache = {}

In [4]:
dataset = pandas.read_csv(path_to_dataset)

### Use code bellow to generate subset of 10MB from original dataset if needed.

In [5]:


dataset = pandas.read_csv(path_to_dataset)

dataset_size_mb = dataset.memory_usage(deep=True).sum() / 1024 / 1024

if dataset_size_mb <= 10:
    subset = dataset
else:
    fraction = 10 / dataset_size_mb
    subset = dataset.sample(frac=fraction, random_state=42)

    subset_size_mb = subset.memory_usage(deep=True).sum() / 1024 / 1024

    while subset_size_mb > 10:
        subset = subset.sample(frac=0.9, random_state=42)
        subset_size_mb = subset.memory_usage(deep=True).sum() / 1024 / 1024

print(f"Original dataset size: {dataset_size_mb:.2f} MB")
print(f"Subset size: {subset_size_mb:.2f} MB")
print(f"Number of rows in original dataset: {len(dataset)}")
print(f"Number of rows in subset: {len(subset)}")

subset.to_csv("subset_data.csv", index=False)
print("Subset saved to 'subset_data.csv'")

Original dataset size: 8148.61 MB
Subset size: 9.06 MB
Number of rows in original dataset: 7483766
Number of rows in subset: 8266
Subset saved to 'subset_data.csv'


### Use code below to convert amount and currency to USD.

In [18]:

@cache
def get_exchange_rate(currency: str):

    url = f'https://v6.exchangerate-api.com/v6/{API_KEY}/latest/{currency.upper()}'
    print(url)

    try:
        response = requests.get(url)
        data = response.json()

        if data['result'] != 'success':
            raise ValueError(f"Error fetching exchange rate: {data.get('error-type', 'Unknown error')}")


        rates = data['conversion_rates']

        usd_rate = rates.get('USD')
        if usd_rate is None:
            print("USD rate not found in response.")
            return None

        return usd_rate
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

def convert_to_usd(row):
    amount = row['amount']
    currency = row['currency']
    timestamp = row['timestamp']


    # TODO: Use timestamp to parse money, and history api. [Needs not free subscription]
    # Format the date for the API ('YYYY-MM-DD')
    # if isinstance(timestamp, str):
    #     try:
    #         timestamp_dt = parser.parse(timestamp)
    #     except ValueError:
    #         return None
    # else:
    #     return None

    rate = get_exchange_rate(currency)
    if rate is None:
        return None

    amount_usd = amount * rate
    return amount_usd


dataset['amount_usd'] = dataset.apply(convert_to_usd, axis=1)
dataset.to_csv("processed_data.csv")

https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/GBP
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/BRL
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/JPY
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/AUD
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/NGN
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/EUR
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/MXN
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/RUB
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/CAD
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/SGD
https://v6.exchangerate-api.com/v6/666fd3adb3ce0cdc27ea2481/latest/USD
